---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open("university_towns.txt") as TOWNS_FILE:
        data = TOWNS_FILE.read()
        
    regions = []
    states = []
    
    for line in data.split('\n'):
        clean_line = line.strip()
        if clean_line == "":
            continue

        if clean_line.endswith('[edit]'):
            current_state = clean_line[:clean_line.index('[')].strip()
        else:
            # If a university is listed it will be in parens. Capture just the city part
            try:
                region_name = clean_line[:clean_line.index('(')].strip()
            except:
                region_name = clean_line.strip()
                
            regions.append(region_name)
            states.append(current_state)
        
    
    return pd.DataFrame({'State': states, 'RegionName': regions})

In [4]:
def load_gdp_data():
    '''Load, clean up and add metadata to the gdp data'''
    df = pd.read_excel("gdplev.xls")
    df = df[["Unnamed: 4", "Unnamed: 6"]]
    df.columns = ["Year and Quarter", "GDP in billions of chained 2009 dollars"]
    
    # There's some garbage headers that we want to get rid of
    df.drop(range(7), inplace=True)
    
    # This is a convenient way to return just 2000q1 and after
    df = df[df["Year and Quarter"].str.startswith('2')]
    df = df.set_index("Year and Quarter")
    
    # Set up negative and "sequential" (ie, double positive/negative) quarters
    df['negQtr'] = df["GDP in billions of chained 2009 dollars"].diff() < 0 
    df['seqQtr'] = df['negQtr'] == df['negQtr'].shift(-1)
    
    return df

gdp_data = load_gdp_data()

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    return gdp_data[gdp_data['negQtr'] & gdp_data['seqQtr']].iloc[0].name

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return gdp_data[
        (gdp_data.index > get_recession_start()) & 
        (gdp_data['negQtr'] == False) & 
        (gdp_data['seqQtr'])
    ].iloc[1].name

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return gdp_data["GDP in billions of chained 2009 dollars"][
        (gdp_data.index >= get_recession_start()) &
        (gdp_data.index <= get_recession_end()) 
    ].argmin()

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    # Cut up the month data, summarize it by quarter, and give it column names in YYYYqQ format
    month_data = df.drop(["RegionID", "RegionName", "State", "Metro", "CountyName", "SizeRank"], axis=1)
    month_data = month_data.groupby(pd.PeriodIndex(month_data.columns, freq='q'), axis=1).mean()
    month_data.columns = ["{}q{}".format(x.year, x.quarter) for x in month_data.columns]
    
    # We only want 2000q1 and later
    month_data = month_data[[c for c in month_data.columns if c.startswith('2')]]
    
    # Add these so we can multi-index on them
    month_data['State'] = df['State'].map(states)
    month_data['RegionName'] = df['RegionName']
    
    return month_data.set_index(['State', 'RegionName'])

In [9]:
def run_ttest():
    '''First create new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    quarter_before_recession = gdp_data[gdp_data.index < recession_start].iloc[-1].name
    
    housing_data = convert_housing_data_to_quarters()
    
    housing_price_before = housing_data[quarter_before_recession]
    housing_price_start = housing_data[recession_start]
    housing_price_bottom = housing_data[recession_bottom]
    
    housing_price_diff = pd.DataFrame((housing_price_start - housing_price_bottom)).dropna()
    housing_price_diff.columns = ['Price Diff']
    
    housing_price_ratio = pd.DataFrame((housing_price_before/housing_price_bottom).dropna())
    housing_price_ratio.columns = ['Price Ratio']
    
    university_data = get_list_of_university_towns()
    university_data['University Town'] = True
    university_data.set_index(['State', 'RegionName'], inplace=True)
    
    merged_diff_data = pd.merge(housing_price_diff, university_data, how='left', left_index=True, right_index=True).fillna(False)
    merged_ratio_data = pd.merge(housing_price_ratio, university_data, how='left', left_index=True, right_index=True).fillna(False)
    
    university_town_housing_diff = merged_diff_data[merged_diff_data['University Town']]['Price Diff'].reset_index(drop=True)
    non_university_town_housing_diff = merged_diff_data[merged_diff_data['University Town'] == False]['Price Diff'].reset_index(drop=True)
    
    university_town_housing_ratio = merged_ratio_data[merged_ratio_data['University Town']]['Price Ratio'].reset_index(drop=True)
    non_university_town_housing_ratio = merged_ratio_data[merged_ratio_data['University Town'] == False]['Price Ratio'].reset_index(drop=True)
    
    (statistic, pvalue) = ttest_ind(university_town_housing_diff, non_university_town_housing_diff)

    different = (pvalue < 0.01)
    u_housing_mean = university_town_housing_ratio.mean()
    nu_housing_mean = non_university_town_housing_ratio.mean()
    
    return (different, pvalue, "university town" if u_housing_mean < nu_housing_mean else "non-university town")